In [ ]:
!pip install arxiv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 6


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [38]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()

In [40]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [41]:
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())

In [42]:
retriever = vectordb.as_retriever()

In [43]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12c37d9c0>, search_kwargs={})

In [44]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"Langsmith_search",
                      "Search for information about langsmith. For any questions about Langsmith, you must use this tool")


In [45]:
retriever_tool.name

'Langsmith_search'

In [46]:
#create Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [47]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [48]:
arxiv.name

'arxiv'

In [49]:
tools = [wiki,arxiv,retriever_tool]

In [50]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/apple/anaconda3/envs/enve/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Langsmith_search', description='Search for information about langsmith. For any questions about Langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x1194eeb90>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.ve

In [51]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo',temperature =0)

In [52]:
# use langchain builtin chat template generic prompt

from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [53]:
# Agents - use LLM model to choose a sequence of actions to take. a Language model is used as a reasoning engine to determine which actions to take and in which order.
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [54]:
##use the agent use agent executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools = tools, verbose=True)

In [55]:
agent_executor.invoke({'input':'Tell me about Langsmith'})



> Entering new AgentExecutor chain...

Invoking: `Langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade Large Language Models (LLM) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith provides observability features, allowing you to analyze traces, configure metrics, dashboards, and alerts based on these. It also offers evaluation capabilities to score application performance and gather human feedback on data. Additionally, LangSmith supports prompt engineering, providing tools for iterating on prompts, version control, and collaboration features to help find the perfect prompt for your application.'}

In [36]:
agent_executor.invoke({'input':'Tell me about machine learning'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can automatically learn and improve from experience without being explicitly programmed. Machine learning is used in various applications such as image recognition, natural language processing, and recommendation systems.

> Finished chain.


{'input': 'Tell me about machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can automatically learn and improve from experience without being explicitly programmed. Machine learning is used in various applications such as image recognition, natural language processing, and recommendation systems.'}